<a href="https://colab.research.google.com/github/maluhuillcaelguera/APP-RaphiAI/blob/main/FASTAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import load_model
model = load_model("mobilenetv2_model.h5")
model.input_shape

(None, 224, 224, 3)

In [ ]:
# ✅ Instalar dependencias necesarias
!pip install fastapi uvicorn nest-asyncio pyngrok

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-52' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

In [ ]:
# ✅ Código de la API FastAPI + carga de modelo entrenado
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
import io

app = FastAPI()

model = load_model("mobilenetv2_model.h5")

class_names = [
    "Potato_early_blight",
    "Potato_healthy",
    "Potato_late_blight",
    "Potato_leafroll_virus",
    "Potato_mosaic_virus"
]

def preprocess_image(file_bytes):
    img = Image.open(io.BytesIO(file_bytes)).convert("RGB")
    img = img.resize((224, 224))  # 👈 Confirmamos tamaño correcto
    img_array = image.img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        img = preprocess_image(contents)
        prediction = model.predict(img)
        predicted_class = class_names[np.argmax(prediction)]
        confidence = float(np.max(prediction))
        return JSONResponse({
            "class": predicted_class,
            "confidence": round(confidence, 4)
        })
    except Exception as e:
        return JSONResponse({"error": str(e)}, status_code=500)


In [ ]:
# ✅ Guardar el archivo app.py
with open("app.py", "w") as f:
    f.write(code)

In [ ]:
# ✅ Copiar el modelo desde Google Drive (asegúrate que el path es correcto)
!cp '/content/drive/MyDrive/MODELOS ENTRENADOS/MOBILNET/mobilenetv2_model.h5' .

In [ ]:
# ✅ Iniciar servidor y ngrok correctamente
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()

# Matar túneles anteriores
ngrok.kill()

# Crear túnel
public_url = ngrok.connect(8000)
print(f"🚀 Tu API está activa en: {public_url}/docs")

# Ejecutar servidor en segundo plano (¡IMPORTANTE!)
!uvicorn app:app --host 0.0.0.0 --port 8000 --reload &

🚀 Tu API está activa en: NgrokTunnel: "https://319339d56401.ngrok-free.app" -> "http://localhost:8000"/docs
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [17801] using StatReload
2025-08-02 04:23:06.999185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754108587.066050   17803 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754108587.088084   17803 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2025-08-02 04:23:16.087288: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
INFO:     Started server process [17803]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     181.224.233.186:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     181.224.233.186:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
INFO:     181.224.233.186:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [17803]
INFO:     Stopping reloader process [17801]
